In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import date, datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
from sklearn.model_selection import KFold
import workalendar
from workalendar.america import Brazil

warnings.filterwarnings('ignore')
np.random.seed(4590)

In [4]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [20]:
hist = pd.read_csv('historical_transactions.csv')
new= pd.read_csv('new_merchant_transactions.csv')

In [21]:
hist = reduce_mem_usage(hist)
new = reduce_mem_usage(new)

Mem. usage decreased to 1749.11 Mb (43.7% reduction)
Mem. usage decreased to 114.20 Mb (45.5% reduction)


In [6]:
hist.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [22]:
for df in [hist,new]:
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)

In [37]:
agg_fun = {'merchant_id': ['count']}
a=hist.groupby(['card_id','merchant_id']).agg(agg_fun)
a.columns = ['_'.join(col).strip() for col in a.columns.values]
a.reset_index(inplace=True)

In [41]:
a

,card_id,merchant_id,merchant_id_count
0,C_ID_00007093c1,M_ID_0379adb435,4
1,C_ID_00007093c1,M_ID_06a8d84366,1
2,C_ID_00007093c1,M_ID_0dabc5a70c,2
3,C_ID_00007093c1,M_ID_109e9d2b1d,2
4,C_ID_00007093c1,M_ID_15446e939c,4
5,C_ID_00007093c1,M_ID_1b440d74fb,1
6,C_ID_00007093c1,M_ID_24e067cb2b,1
7,C_ID_00007093c1,M_ID_2efbd71a00,1
8,C_ID_00007093c1,M_ID_3c6556f2dd,1
9,C_ID_00007093c1,M_ID_490fc654fb,3


In [39]:
agg_fun = {'merchant_id': ['count']}
b=new.groupby(['card_id','merchant_id']).agg(agg_fun)
b.columns = ['_'.join(col).strip() for col in b.columns.values]
b.reset_index(inplace=True)

In [40]:
b

,card_id,merchant_id,merchant_id_count
0,C_ID_00007093c1,M_ID_00a6ca8a8a,1
1,C_ID_00007093c1,M_ID_08f01305af,1
2,C_ID_0001238066,M_ID_00a6ca8a8a,2
3,C_ID_0001238066,M_ID_0bb734e74a,1
4,C_ID_0001238066,M_ID_235e546dcc,1
5,C_ID_0001238066,M_ID_4635824091,1
6,C_ID_0001238066,M_ID_51b4a616bb,1
7,C_ID_0001238066,M_ID_6b8228d6a6,1
8,C_ID_0001238066,M_ID_7d8102bb34,1
9,C_ID_0001238066,M_ID_84f1ff55b9,1


In [54]:
c=pd.merge(a,b,on=['card_id','merchant_id'],how='outer')

In [55]:
c.shape

(12928941, 4)

In [56]:
c=c.fillna(0)

In [58]:
c['merchant_id_count']=c['merchant_id_count_x']+c['merchant_id_count_y']

In [63]:
d=c[['card_id','merchant_id','merchant_id_count']].copy()

In [70]:
agg_fun = {'merchant_id_count': ['max']}
e=d.groupby(['card_id']).agg(agg_fun)
e.columns = ['_'.join(col).strip() for col in e.columns.values]
e.reset_index(inplace=True)

In [82]:
e.rename(columns={'merchant_id_count_max':'merchant_id_count'},inplace=True)

In [77]:
c.drop_duplicates(subset=['card_id','merchant_id_count'],keep='first',inplace=True)

In [85]:
final=pd.merge(e,c,on=['card_id','merchant_id_count'],how='left')

In [91]:
final.rename(columns={'merchant_id':'merchant_id_max'},inplace=True)

In [92]:
final.head()

,card_id,merchant_id_max,merchant_id_count
0,C_ID_00007093c1,M_ID_9400cf2342,42.0
1,C_ID_0001238066,M_ID_d17aabd756,24.0
2,C_ID_0001506ef0,M_ID_b1fc88154d,13.0
3,C_ID_0001793786,M_ID_923d57de8d,17.0
4,C_ID_000183fdda,M_ID_f9cfe0a43b,13.0


In [88]:
final=final[['card_id','merchant_id','merchant_id_count']]

In [93]:
final.to_csv('national holidays/card_merchant.csv',index=False)

In [79]:
c.shape

(1981904, 5)

In [83]:
e.head()

,card_id,merchant_id_count
0,C_ID_00007093c1,42.0
1,C_ID_0001238066,24.0
2,C_ID_0001506ef0,13.0
3,C_ID_0001793786,17.0
4,C_ID_000183fdda,13.0


In [75]:
e.head()

,card_id,merchant_id_count_max
0,C_ID_00007093c1,42.0
1,C_ID_0001238066,24.0
2,C_ID_0001506ef0,13.0
3,C_ID_0001793786,17.0
4,C_ID_000183fdda,13.0


In [84]:
c.head()

,card_id,merchant_id,merchant_id_count_x,merchant_id_count_y,merchant_id_count
0,C_ID_00007093c1,M_ID_0379adb435,4.0,0.0,4.0
1,C_ID_00007093c1,M_ID_06a8d84366,1.0,0.0,1.0
2,C_ID_00007093c1,M_ID_0dabc5a70c,2.0,0.0,2.0
9,C_ID_00007093c1,M_ID_490fc654fb,3.0,0.0,3.0
11,C_ID_00007093c1,M_ID_57df19bf28,28.0,0.0,28.0
